# Visualizing Samples and Labels of the SPEED Dataset

This notebook helps to inspect the SPEED dataset. You can see samples from the dataset, with the corresponding ground truth labels visualized as projected axes.

In [ ]:
import numpy as np
import json
import os
from PIL import Image, ImageDraw
from matplotlib import pyplot as plt
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing import image as keras_image
#%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D
import mpl_toolkits.mplot3d as a3
import matplotlib.colors as colors
from matplotlib.collections import PolyCollection

class Camera:

    """" Utility class for accessing camera parameters. """

    fx = 0.0176  # focal length[m]
    fy = 0.0176  # focal length[m]
    nu = 1920  # number of horizontal[pixels]
    nv = 1200  # number of vertical[pixels]
    ppx = 5.86e-6  # horizontal pixel pitch[m / pixel]
    ppy = ppx  # vertical pixel pitch[m / pixel]
    fpx = fx / ppx  # horizontal focal length[pixels]
    fpy = fy / ppy  # vertical focal length[pixels]
    k = [[fpx,   0, nu / 2],
         [0,   fpy, nv / 2],
         [0,     0,      1]]
    K = np.array(k)


def process_json_dataset(root_dir):
    with open(os.path.join(root_dir, 'train.json'), 'r') as f:
        train_images_labels = json.load(f)

    with open(os.path.join(root_dir, 'test.json'), 'r') as f:
        test_image_list = json.load(f)

    with open(os.path.join(root_dir, 'real_test.json'), 'r') as f:
        real_test_image_list = json.load(f)

    partitions = {'test': [], 'train': [], 'real_test': []}
    labels = {}

    for image_ann in train_images_labels:
        partitions['train'].append(image_ann['filename'])
        labels[image_ann['filename']] = {'q': image_ann['q_vbs2tango'], 'r': image_ann['r_Vo2To_vbs_true']}

    for image in test_image_list:
        partitions['test'].append(image['filename'])

    for image in real_test_image_list:
        partitions['real_test'].append(image['filename'])

    return partitions, labels


def quat2dcm(q):

    """ Computing direction cosine matrix from quaternion, adapted from PyNav. """

    # normalizing quaternion
    q = q/np.linalg.norm(q)

    q0 = q[0]
    q1 = q[1]
    q2 = q[2]
    q3 = q[3]

    dcm = np.zeros((3, 3))

    dcm[0, 0] = 2 * q0 ** 2 - 1 + 2 * q1 ** 2
    dcm[1, 1] = 2 * q0 ** 2 - 1 + 2 * q2 ** 2
    dcm[2, 2] = 2 * q0 ** 2 - 1 + 2 * q3 ** 2

    dcm[0, 1] = 2 * q1 * q2 + 2 * q0 * q3
    dcm[0, 2] = 2 * q1 * q3 - 2 * q0 * q2

    dcm[1, 0] = 2 * q1 * q2 - 2 * q0 * q3
    dcm[1, 2] = 2 * q2 * q3 + 2 * q0 * q1

    dcm[2, 0] = 2 * q1 * q3 + 2 * q0 * q2
    dcm[2, 1] = 2 * q2 * q3 - 2 * q0 * q1

    return dcm

def pointInTriangle(t, p):
    #https://stackoverflow.com/questions/2049582/how-to-determine-if-a-point-is-in-a-2d-triangle
    a = 0.5 *(-t[1][1]*t[2][0] + t[0][1]*(-t[1][0] + t[2][0]) + t[0][0]*(t[1][1] - t[2][1]) + t[1][0]*t[2][1]);
    s = 1/(2*a)*(t[0][1]*t[2][0] - t[0][0]*t[2][1] + (t[2][1] - t[0][1])*p[0] + (t[0][0] - t[2][0])*p[1]);
    u = 1/(2*a)*(t[0][0]*t[1][1] - t[0][1]*t[1][0] + (t[0][1] - t[1][1])*p[0] + (t[1][0] - t[0][0])*p[1]);
    return s > 0 and u > 0 and (1-s-u) > 0
    
class Plane:
    def __init__(self, points):
        if len(points) != 3:
            raise ValueError("Plane always consists of three points")
        
        self.points = np.asarray(points)
        n = np.cross(points[1] - points[0], points[2] - points[0])
        self.normal = n / np.linalg.norm(n, 2)
    
    def intersect(self, v):
        ndotu = self.normal.dot(v)
        if abs(ndotu) < 1e-6:
            raise ValueError("Line is parallel to plane")
 
        return -self.points[0] - (self.normal.dot(-self.points[0]) / ndotu) * v + self.points[0]
    
    def intersects(self, v):
        # First calculate intersection point of vector and this plane:
        try:
            intersection = self.intersect(v)
        except ValueError:
            # The vector is parallel to the line, so always return false (could be completely on it or completely off)
            return False
        
        # We have a rotated 3D plane (i.e. z coordinates are level) and want to remove the
        # z coordinates while keeping relations (i.e. project 3D plane to xy-plane)
        # https://stackoverflow.com/questions/1023948/rotate-normal-vector-onto-axis-plane
        zAxisNew = self.normal
        xAxisOld = np.array([1,0,0])
        if np.array_equal(np.absolute(zAxisNew), xAxisOld):
            # the old x axis cannot be the same as the normal (the new z axis) since then the
            # coordinate system is perpendicular to the xy plane. Therefore change x and z then
            xAxisOld = np.array([0,0,1])
        yAxisOld = np.array([0,1,0])
        yAxisNew = np.cross(xAxisOld, zAxisNew)
        xAxisNew = np.cross(zAxisNew, yAxisNew)
        yAxisNew /= np.linalg.norm(yAxisNew, 2)
        xAxisNew /= np.linalg.norm(xAxisNew, 2)
        projected2dtriangle = np.asarray([[p.dot(xAxisNew), p.dot(yAxisNew)] for p in self.points])
        # Now we know the 2d projection of the points of the polygon. Also project the 3d intersection point to the same plane
        projected2dpoint = np.asarray([intersection.dot(xAxisNew), intersection.dot(yAxisNew)])
        return intersection, pointInTriangle(projected2dtriangle, projected2dpoint)

def getSatelliteModel():
    b = 0.6
    a = 0.75
    d = 0.8
    c = 0.32

    #     0         1      
    #     +---a-----+
    #  d-/|   u    /|-c
    # 3 +---------+ | 2
    #   |w| y  z  |x|     (y: front, z: back)
    # 4 | +-------|-+ 5
    #   |/ v (0,0)|/-b 
    # 7 +---------+ 6
    # reference points in satellite frame for drawing axes
    return np.array([
        [-a / 2,  d / 2, c], # 0
        [ a / 2,  d / 2, c], # 1
        [ a / 2, -d / 2, c], # 2
        [-a / 2, -d / 2, c], # 3
        [-a / 2,  b / 2, 0], # 4
        [ a / 2,  b / 2, 0], # 5
        [ a / 2, -b / 2, 0], # 6
        [-a / 2, -b / 2, 0]  # 7
    ]), np.array([
        [0, 1, 2], [0, 3, 2], # u
        [4, 5, 6], [4, 7, 6], # v
        [0, 3, 7], [0, 4, 7], # w
        [1, 2, 6], [1, 5, 6], # x
        [3, 2, 6], [3, 7, 6], # y
        [0, 1, 5], [0, 4, 5], # z
    ])

def project(q, r, plot=False):
    """ Projecting points to image frame to draw axes """
    model_coordinates, cube_polygon_indices = getSatelliteModel()
    p_axes = np.ones((model_coordinates.shape[0], model_coordinates.shape[1] + 1))
    p_axes[:,:-1] = model_coordinates
    points_body = np.transpose(p_axes)

    # transformation to camera frame
    pose_mat = np.hstack((np.transpose(quat2dcm(q)), np.expand_dims(r, 1)))
    p_cam = np.dot(pose_mat, points_body)

    # Indices of points describing 3 point triangles of the cube
    # No point should intersect any of these triangles to be visible in the camera

    if plot:
        fig = plt.figure()
        ax = Axes3D(fig)

    points_camera_t = p_cam.transpose()
    points_camera_collision_indices = []
    for polygon_indices in cube_polygon_indices:
        points_polygon = points_camera_t[polygon_indices]
        plane = Plane(points_polygon)

        if plot:
            tri = a3.art3d.Poly3DCollection([plane.points], alpha=0.2)
            tri.set_color([1,0,0])
            tri.set_edgecolor('k')
            ax.add_collection3d(tri)

        for i, p in enumerate(points_camera_t):
            intersection, intersects = plane.intersects(p)
            if(intersects):
                # The vector between camera origin and cube vertice intersects any of the 12 cube polygons.
                # There are two border cases to check:
                # 1) Sometimes an actual vertice intersects a neighboring polygon
                # 2) The vector between camera and point intersects a polygon that actually is behind the point
                dist_intersection = np.linalg.norm(intersection, 2)
                dist_point = np.linalg.norm(p, 2)
                if abs(dist_intersection - dist_point) > 0.01 and dist_intersection < dist_point and not i in points_camera_collision_indices:
                    points_camera_collision_indices.append(i)
                    if plot:
                        ax.scatter([intersection[0]], [intersection[1]], [intersection[2]])

    visible_points = np.ones(len(p_axes), dtype=bool)
    visible_points[points_camera_collision_indices] = False

    if plot:
        for p in points_camera_t[visible_points]:
            ax.plot([0, p[0]], [0, p[1]], [0, p[2]])

        #ax.set_xlim(-1, 1)
        #ax.set_ylim(-1, 1)
        #ax.set_zlim(5, 7)
        ax.autoscale()
        ax.set_xlabel('X axis')
        ax.set_ylabel('Y axis')
        ax.set_zlabel('Z axis')

        plt.show()

    p_cam = points_camera_t.transpose()

    # getting homogeneous coordinates
    points_camera_frame = p_cam / p_cam[2]
    # projection to image plane
    points_image_plane = Camera.K.dot(points_camera_frame)

    x, y = (points_image_plane[0], points_image_plane[1])
    return x, y, visible_points

def projectOrientationSystem(q, r):

    """ Projecting points to image frame to draw axes """

    # reference points in satellite frame for drawing axes
    p_axes = np.array([[0, 0, 0, 1],
                       [1, 0, 0, 1],
                       [0, 1, 0, 1],
                       [0, 0, 1, 1]])
    points_body = np.transpose(p_axes)

    # transformation to camera frame
    pose_mat = np.hstack((np.transpose(quat2dcm(q)), np.expand_dims(r, 1)))
    p_cam = np.dot(pose_mat, points_body)

    # getting homogeneous coordinates
    points_camera_frame = p_cam / p_cam[2]

    # projection to image plane
    points_image_plane = Camera.K.dot(points_camera_frame)

    x, y = (points_image_plane[0], points_image_plane[1])
    return x, y


class SatellitePoseEstimationDataset:

    """ Class for dataset inspection: easily accessing single images, and corresponding ground truth pose data. """

    def __init__(self, root_dir='/datasets/speed_debug'):
        self.partitions, self.labels = process_json_dataset(root_dir)
        self.root_dir = root_dir

    def get_image(self, i=0, split='train'):

        """ Loading image as PIL image. """

        img_name = self.partitions[split][i]
        img_name = os.path.join(self.root_dir, 'images', split, img_name)
        image = Image.open(img_name).convert('RGB')
        return image

    def get_pose(self, i=0):

        """ Getting pose label for image. """

        img_id = self.partitions['train'][i]
        q, r = self.labels[img_id]['q'], self.labels[img_id]['r']
        return q, r

    def visualize(self, i, partition='train', ax=None):

        """ Visualizing image, with ground truth pose with axes projected to training image. """

        if ax is None:
            ax = plt.gca()
        img = self.get_image(i)
        ax.imshow(img)

        # no pose label for test
        if partition == 'train':
            q, r = self.get_pose(i)
            xa, ya = projectOrientationSystem(q, r)
            ax.arrow(xa[0], ya[0], xa[1] - xa[0], ya[1] - ya[0], color='r')
            ax.arrow(xa[0], ya[0], xa[2] - xa[0], ya[2] - ya[0], color='g')
            ax.arrow(xa[0], ya[0], xa[3] - xa[0], ya[3] - ya[0], color='b')

        return


In [ ]:
dataset_root_dir = './speed'
dataset = SatellitePoseEstimationDataset(root_dir=dataset_root_dir)

In [ ]:
rows = 4
cols = 2
#%matplotlib notebook
%matplotlib inline
def drawBlob(img, pos, size=3, color=[255, 0, 0]):
    for y in range(pos[1] - size, pos[1] + size):
        for x in range(pos[0] - size, pos[0] + size):
            img[y][x] = color

# 1) 8 Kantenpunkte bestimmen
# 2) 8 entspr. Flächen bestimmenn
# 3) 8 Vektor zwischen Kameraprojektion und 3D Punkt bestimmen
# 4) Überprüfen, ob die 8 Vektoren irgendeine der 8 Flächen durchschneiden. Wenn ja: Punkt verwerfen!

for i in range(0, 1):
    img = np.array(dataset.get_image(i))
    q, r = dataset.get_pose(i)
    xa, ya, visible = project(q, r)
    for x, y, v in zip(xa, ya, visible):
        if v and x >= 0.0 and y >= 0.0 and x <= Camera.nu and y <= Camera.nv:
            drawBlob(img, (int(x), int(y)))
    
    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    plt.show()


In [ ]:
class KerasDataGenerator(Sequence):

    """ DataGenerator for Keras to be used with fit_generator (https://keras.io/models/sequential/#fit_generator)"""

    def __init__(self, label_list, speed_root, label_size, batch_size=32, dim=(224, 224), shuffle=True):

        # loading dataset
        self.image_root = os.path.join(speed_root, 'images', 'train')

        # Initialization
        self.dim = dim
        self.batch_size = batch_size
        self.labels = self.labels = {label['filename']: {'q': label['q_vbs2tango'], 'r': label['r_Vo2To_vbs_true']}
                                     for label in label_list}
        self.list_IDs = [label['filename'] for label in label_list]
        self.shuffle = shuffle
        self.label_size = label_size
        self.indexes = None
        self.on_epoch_end()

    def __len__(self):

        """ Denotes the number of batches per epoch. """

        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):

        """ Generate one batch of data """

        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        return self.__data_generation(list_IDs_temp)

    def on_epoch_end(self):

        """ Updates indexes after each epoch """

        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def drawBlob(self, img, pos, sigma=3):
        # https://github.com/NVlabs/Deep_Object_Pose/blob/master/src/training/train.py#L851
        w = int(sigma*3)
        if pos[0]-w>=0 and pos[0]+w<img.shape[0] and pos[1]-w>=0 and pos[1]+w<img.shape[1]:
            for i in range(int(pos[0])-w, int(pos[0])+w):
                for j in range(int(pos[1])-w, int(pos[1])+w):
                    img[i,j] = np.exp(-(((i - pos[0])**2 + (j - pos[1])**2)/(2*(sigma**2))))

    def __data_generation(self, list_IDs_temp):

        """ Generates data containing batch_size samples """

        # Initialization
        imgs = np.empty((self.batch_size, *self.dim, 1))
        masks = np.zeros((self.batch_size, *self.dim, 8), dtype=np.float)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            img_path = os.path.join(self.image_root, ID)
            img = keras_image.load_img(img_path, target_size=self.dim, color_mode = "grayscale")
            imgs[i] = keras_image.img_to_array(img)

            q, r = self.labels[ID]['q'], self.labels[ID]['r']
            xa, ya, visibles = project(q, r)
            for j, (x, y, visible) in enumerate(zip(xa, ya, visibles)):
                x /= Camera.nu
                y /= Camera.nv
                if visible and x >= 0.0 and y >= 0.0 and x <= 1.0 and y <= 1.0:
                    x_s, y_s = int(x * self.dim[1]), int(y * self.dim[0])
                    self.drawBlob(masks[i][...,j], (x_s, y_s), self.label_size)

        return imgs, masks

In [ ]:
# Setting up parameters
params = {'dim': (480, 640),
          'batch_size': 8,
          'label_size': 3,
          'shuffle': True}

# Loading and splitting dataset
with open(os.path.join(dataset_root_dir, 'train' + '.json'), 'r') as f:
    label_list = json.load(f)
train_labels = label_list[:int(len(label_list)*.8)]
validation_labels = label_list[int(len(label_list)*.8):]

# Data generators for training and validation
training_generator = KerasDataGenerator(train_labels, dataset_root_dir, **params)
validation_generator = KerasDataGenerator(validation_labels, dataset_root_dir, **params)


In [ ]:
for imgs, masks in training_generator:
    print(imgs.shape, masks.shape)
    for img, mask in zip(imgs, masks):        
        # plot with various axes scales
        plt.figure(figsize=(20, 20))

        plt.subplot(121)
        plt.imshow(img.astype(np.uint8)[...,0], cmap='gray')

        plt.subplot(122)
        plt.imshow(mask[...,0], cmap='gray')
        plt.show()
        break
    break

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose, Dense, BatchNormalization, Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import backend as K

In [ ]:
def jaccard_distance_loss(y_true, y_pred, smooth=100):
    """
    Jaccard = (|X & Y|)/ (|X|+ |Y| - |X & Y|)
            = sum(|A*B|)/(sum(|A|)+sum(|B|)-sum(|A*B|))
    
    The jaccard distance loss is usefull for unbalanced datasets. This has been
    shifted so it converges on 0 and is smoothed to avoid exploding or disapearing
    gradient.
    
    Ref: https://en.wikipedia.org/wiki/Jaccard_index
    
    @url: https://gist.github.com/wassname/f1452b748efcbeb4cb9b1d059dce6f96
    @author: wassname
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac) * smooth

def soft_dice_loss(y_true, y_pred, epsilon=1e-6): 
    ''' 
    Soft dice loss calculation for arbitrary batch size, number of classes, and number of spatial dimensions.
    Assumes the `channels_last` format.
  
    # Arguments
        y_true: b x X x Y( x Z...) x c One hot encoding of ground truth
        y_pred: b x X x Y( x Z...) x c Network output, must sum to 1 over c channel (such as after softmax) 
        epsilon: Used for numerical stability to avoid divide by zero errors
    
    # References
        V-Net: Fully Convolutional Neural Networks for Volumetric Medical Image Segmentation 
        https://arxiv.org/abs/1606.04797
        More details on Dice loss formulation 
        https://mediatum.ub.tum.de/doc/1395260/1395260.pdf (page 72)
        
        Adapted from https://github.com/Lasagne/Recipes/issues/99#issuecomment-347775022
    '''
    
    # skip the batch and class axis for calculating Dice score
    axes = tuple(range(1, len(y_pred.shape)-1)) 
    numerator = 2. * K.sum(y_pred * y_true, axes)
    denominator = K.sum(K.square(y_pred) + K.square(y_true), axes)
    
    return 1 - K.mean(numerator / (denominator + epsilon)) # average over classes and batch

def focal_loss(target, output, gamma=2):
    # https://github.com/keras-team/keras/issues/6261#issuecomment-358826560
    output /= K.sum(output, axis=-1, keepdims=True)
    eps = K.epsilon()
    output = K.clip(output, eps, 1. - eps)
    return -K.sum(K.pow(1. - output, gamma) * target * K.log(output), axis=-1)

def conv_norm(inp, filters, conv=Conv2D, kernel_size=3):
    c = conv(filters=filters, kernel_size=kernel_size, padding='same')(inp)
    c = BatchNormalization()(c)
    return LeakyReLU(0.1)(c)

def encode(inp, filters):
    c = conv_norm(inp, filters)
    c = conv_norm(c, filters)
    c = conv_norm(c, filters)
    p = MaxPooling2D(pool_size=2)(c)
    return c, p

def decode(inp, shortcut, filters):
    up = concatenate([Conv2DTranspose(filters, 2, strides=2, padding='same')(inp), shortcut], axis=3)
    c = BatchNormalization()(up)
    c = conv_norm(c, filters)
    return conv_norm(c, filters)

filters_encode_decode = [16,32,32,64,128]
filters_middle = [256, 256, 256]

layers = [Input(params['dim'] + (1,))]
for i, filters in enumerate(filters_encode_decode):
    c, p = encode(layers[-1], filters)
    layers.append(c)
    layers.append(p)

for i, filters in enumerate(filters_middle):
    layers.append(conv_norm(layers[-1], filters))

for i, filters in enumerate(reversed(filters_encode_decode)):
    layers.append(decode(layers[-1], layers[((len(filters_encode_decode) - i) * 2) - 1], filters))

layers.append(Conv2D(8, (1, 1), activation='sigmoid')(layers[-1]))

model = Model(inputs=[layers[0]], outputs=[layers[-1]])
model.compile(optimizer = RMSprop(lr=1e03), loss = focal_loss, metrics=['accuracy'])
model.summary()


In [ ]:
current_model = "m1.h5"
checkpoint = ModelCheckpoint(current_model,save_best_only=True, verbose=1, monitor="val_loss")
reduce = ReduceLROnPlateau(factor=0.1, patience=5, monitor='val_loss')
earlyStopping = EarlyStopping(patience=20, verbose=1,monitor="val_loss")
history = model.fit_generator(
    generator=training_generator,
    validation_data=validation_generator,
    use_multiprocessing=True, # Only works if training data is loaded into RAM from HDF
    workers=8,
    callbacks=[earlyStopping, checkpoint, reduce],
    epochs=10000
)

In [ ]:
for imgs, masks in training_generator:
    for img, mask in zip(imgs, masks):
        pred = model.predict(np.asarray([img]))
        
        # plot with various axes scales
        plt.figure(figsize=(20, 20))

        plt.subplot(121)
        plt.imshow(img.astype(np.uint8)[...,0], cmap='gray')

        plt.subplot(122)
        plt.imshow(pred[0][...,2])

        plt.show()

    break

In [ ]:
#!/usr/bin/env python

index = 0

import cv2
import numpy as np
 
# Read Image
img = np.array(dataset.get_image(index))
q, r = dataset.get_pose(index)
xa, ya, visible = project(q, r)
size = img.shape

model_points_all, _ = getSatelliteModel()
image_points_all = np.stack((xa, ya), axis=1)

model_points = model_points_all[visible]
image_points = image_points_all[visible]
print(visible)
print(model_points)
print(image_points)

In [ ]:
from scipy.spatial.transform import Rotation as R
 
(success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, image_points, Camera.K, None)

print(q, r)
rot = np.zeros((3, 3), dtype=np.float)
#cv2.Rodrigues(rotation_vector, rot)
print(R.from_rotvec(rotation_vector[...,0]).as_quat(), translation_vector[...,0])
#print(R.from_dcm(rot).as_euler('zyx', degrees=True), R.from_dcm(rot).as_euler('zyx', degrees=True))

In [ ]:
rows = 4
cols = 2

fig, axes = plt.subplots(rows, cols, figsize=(12, 12))
for i in range(rows):
    for j in range(cols):
        dataset.visualize(i * rows + j, ax=axes[i][j])
        axes[i][j].axis('off')
fig.tight_layout() 

In [ ]:
# Project a 3D point (0, 0, 1000.0) onto the image plane.
# We use this to draw a line sticking out of the nose
 
 
(nose_end_point2D, jacobian) = cv2.projectPoints(np.array([(1.0, 1.0, 1.0)]), rotation_vector, translation_vector, Camera.K, None)
 
for p in image_points:
    cv2.circle(img, (int(p[0]), int(p[1])), 5, (0,0,255), -1)
 
 
p1 = ( int(image_points[0][0]), int(image_points[0][1]))
p2 = ( int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]))
 
cv2.line(img, p1, p2, (255,0,0), 2)
 
# Display image
plt.figure(figsize=(20,20))
plt.imshow(img)
plt.show()